<a href="https://colab.research.google.com/github/sun-in-universe/blue_dot/blob/main/Initialization_%EC%B4%88%EA%B8%B0%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
- 초기 가중치 지정 : 가중치를 초기화하는 것이 학습 과정을 도움
- random, zeros, Xavier initialization

* 기울기 하강 수렴을 빠르게 한다.
* 경사 하강법이 더 낮은 훈련 (및 일반화) 오차로 수렴할 확률을 높이다
= 경사하강법을 빠르게, 적은 오차로 만드는 것이 좋은 Initialization !

<1> wegiht = zero initialization
- 모든 weight = 0, 모든 결론이 zero가 된다.

결론 !
(1) 가중치를 랜덤하게 설정해서 대칭성을 없애야한다.
(2) b를 0으로 초기화하는 것은 괜찮다.

<2> weight = random initializaiton
- large random-valued weight -> 손실을 높임
- log(AL) = log(0)이면 손실은 무한대로 증폭
- 초기화를 잘못 설정하면 기울기 소실 or 폭발 문제가 일어나고-> 알고리즘의 최적화를 방해한다.
- 오랫동안 훈련을 시키면 더 나은 결과를 낼 수 있긴 하지만, 매우 큰 random 수로 초기화하는 것은 최적화를 느리게 한다.

결론 !
(1) very large random values로 초기화 하면 안된다.
(2) small rnadom values가 낫지만, 얼마나 작아야 하냐는 여전히 의문점으로 남는다.

<3> weight = Xavier initialization
- sqrt(2./layers_dims[l-1])



In [ ]:
#Initialize_Xavier_parameter

def initialize_Xavier_parameter(layer_dims):
  """
  Arguments:
  layer_dims -- python array contining the dimensions of each layer in our network ex) [2, 5, 4... ]

  Returns:
  parameters -- python dictionary containing your parmaeters "W1", "b1"... "WL", "bL":
  Wl = weight matrix of shape(layer_dims[l], layer_dims[l-1])
  b1 = bias vector of shape(layer_dims[l], 1)
  """

  np.random.seed(3)
  parameters = {}
  L = len(layer_dims)

  for i in range(1, L):
    parameters["W" + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1]) * np.sqrt(2. / layers_dims[l-1])
    parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters


In [ ]:
#Forward propagation

def linear_forward(A, W, b):
  """
  Arguments:
  A--activations from previous layer(or input data)
  W--weights matrix: numpy array of shape
  b--bias vector

  Returns:
  Z -- the input of the activation function, also called pre-activation parameter
  cache -- a python tuple containing "A", "W", "b"
  """

  Z = np.dot(W, A) + b
  cache = (A, W, b)

  return Z, cache


def sigmoid(z):
  cache = (A, W, b)
  return 1 / (1 + np.exp(-z)), cache

def relu(z):
  cache = (A, W, b)
  return max(0, z), cache


def linear_activation_forward(A_prev, W, b, activation):
  if activation == "sigmoid":
    Z, linear_cache = linear_forward(A_prev, W, b)
    A, activation_cache = sigmoid(Z)
  elif activation == "relu":
    Z, linear_cache = linear_forward(A_prev, W, b)
    A, activation_cache = relu(Z)

  return A, cache

def forward_propagation(X, parameters):
  """
  implement forward propagation for the [Linear->Relu]*(L-1)->Linear->Sigmoid computation

  Arguments:
  X -- data
  parameters -- out of initialize_parameters_deep()

  Returns:
  AL -- activation value from the output(last) layer
  caches -list of caches containing
  """

  caches = []
  A = X
  L = len(parameters) // 2

  for i in range(1, L):
    A_prev = A
    A, cache = linear_activation_forward(A_prev, parameters[f'W{l}'], parameters[f'b{l}'], "relu")
    caches.append(caches)

  AL, caches = linear_activation_forward(A, parameters[f'W{l}'], parameters[f'b{l}'], "sigmoid")

  return AL, caches

In [ ]:
#compute cost

def compute_cost(AL, Y):
  """

  Arguments:
  AL -- probability vector corresponding to your label predictions, shape(1, numeber of examples)
  Y -- true ""label"" vector, shape(1, numer of examples)

  Returns:
  cost -- corss-entropy cost
  """

  m = Y.shape[1] #Y의 갯수

  cost = -(1/m) * np.sum((Y * np.log(AL) + ((1-Y) * np.log(1-AL))))
  dZ = AL - Y  # This is the derivative of the cost with respect to Z

  cost = np.squeeze(cost)

  return cost, dZ

In [ ]:
# Backward propagation

def linear_backward(dZ, cache):
  """
  Arguments:
  dZ -- Gradient of the cost with respect to the linear output
  cache -- tuple of values(A_prev, W, b) coming from the forward propagation (A, W, b)

  Returns:
  dA_prev -- Gradient of the cost with respect to the activation, same shape as A_prev
  dW -- Gradient of the cost with respect to W
  db -- Gradient of the cost with respoect to b
  """"

  A_prev, W, b = cache
  m = A_prev.shape[1]

  dw = (1/m) * np.dot(dZ, A_prev.T)
  db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
  dA_prev = np.dot(W.T, dZ)

  return dA_prev, dW, db

def relu_backward(dA, activation_cache):
    Z = activation_cache
    dZ = np.array(dA, copy=True)  # Initialize dZ with dA, then set dZ=0 where Z<=0
    dZ[Z <= 0] = 0
    return dZ

def sigmoid_backward(dA, activation_cache):
    Z = activation_cache
    A = 1 / (1 + np.exp(-Z))
    dZ = dA * A * (1 - A)
    return dZ


def linear_activation_backward(dA, cache, activation):
  """
  Arguments:
  dA -- post activation gradient for current layer
  cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
  activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

  Returns:
  dA_prev -- Gradient of the cost with respect to the activation
  dW -- Gradient of the cost with respect to W
  db -- Gradient of the cost with respoect to b
  """

  linear_cache, activation_cache = cache

  if activation == "relu":
    dZ = relu_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

  elif activation == "sigmoid":
    dZ = sigmoid_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

  return dA_prev, dW, db

def backward_propagation(AL, Y, caches):
  """
  implement the backward propagation for the [Linear->Relu] * (L-1) -> Linear -> Sigmoid group

  Arguments:
  AL -- probability vector, out of the forward propagation
  Y-- true "label" vector
  caches -- list of caches containing => every caches of linear_activation_forward with "relu" and "sigmoid"

  Returns:
  grads -- A dictionary with the gradients
           grads["dA" + str(l)] =
           grads["dW" + str(l)] =
           grads["db" + str(l)] =
  """

  grads = {}
  L = len(caches)
  m = AL.shape[1]
  Y = Y.reshape(AL.shape)

  #initializaing the backpropagation
  dAL = -(np.divdie(Y, AL)-np.divde(1-Y, 1-AL))

  #Lth layer(sigmoid->Linear) gradients. Inputs: dAL, current cache, Outputs: grads["dAL-1"], grads["dWL"], grad["dbL"]
  current_cache = caches[L-1]
  dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, "sigmoid")
  grads["dA" + str(L-1)] = dA_prev_temp
  grads["dW" + str(L)] = dW_temp
  grads["db" + str(L)] = db_temp

  for l in reversed(range(L-1)):
    current_cache = caches[l]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, "relu")
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l + 1)] = dW_temp
    grads["db" + str(l + 1)] = db_temp

  return grads


current_cache = caches[L-1]: 이 줄은 캐시 리스트(caches)에서 마지막 층(L번째 층)의 캐시된 값들 (linear_cache 및 activation_cache)을 current_cache에 초기화합니다.

dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, "sigmoid"): 이 줄은 마지막 (L번째) 층에 대한 기울기를 계산합니다. linear_activation_backward 함수를 사용하며 다음과 같은 인자들을 사용합니다:

dAL은 현재 (L번째) 층의 활성화에 대한 비용의 기울기입니다.
current_cache는 현재 층에 대한 캐시된 값들을 포함합니다.
"sigmoid"는 이 층에서 시그모이드 활성화 함수를 사용함을 나타냅니다.
계산된 기울기들 (dA_prev_temp, dW_temp, db_temp)은 그런 다음 적절한 키로 grads 사전에 저장됩니다. 이는 마지막 층에 대한 기울기를 나타냅니다.

코드는 L-1번째부터 역방향으로 순회하는 루프에 들어갑니다. 이때 L은 네트워크의 층 수를 나타냅니다.

current_cache = caches[l]: 이 줄은 현재 층 l에 대한 캐시된 값들을 current_cache에 초기화합니다.

dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, "relu"): 이 줄은 현재 층 l에 대한 기울기를 계산합니다. linear_activation_backward 함수를 사용하며 다음과 같은 인자들을 사용합니다:

grads["dA" + str(l + 1)]은 현재 (l+1)번째 층의 활성화에 대한 비용의 기울기입니다. 이전 층에 대한 기울기입니다.
current_cache는 현재 층에 대한 캐시된 값들을 포함합니다.
"relu"는 이 층에서 ReLU 활성화 함수를 사용함을 나타냅니다.
계산된 기울기들 (dA_prev_temp, dW_temp, db_temp)은 그런 다음 적절한 키로 grads 사전에 저장됩니다. 이는 현재 층 l에 대한 기울기를 나타냅니다.

In [1]:
# update parameters

def update_parameters(parameters, grads, learning_rate):
  """
  Arguments:
  parameters -- python dictionary containing your parameters
  grads -- also dict containing gradients, output of backpropagation

  Returns:
  parameters -- also dict containing your updated parameters

  """
  parameters = parameters.copy()
  L = len(parameters) // 2

  for l in range(L):
    parameters["W" + str(l)] -= learning_rate * grads["dW" + str(l)]
    parameters["b" + str(l)] -= learning_rate * grads["db" + str(l)]

  return parameters


1. 모델 만들기
2. 라이브러리 임포트
3. numpy 임의성 조정 with random_seed
4. 데이터 셋 가져오기
5. train, val, test set으로 나누기
6. initializaing parameters
7. forward propagation
8. cost computation
9. parameters update
10. train_nn 모델 최종 만들기 및 테스트

In [ ]:
# 신경망을 학습시키는 함수
def train_nn(X_train, Y_train, X_test, Y_test, neurons_per_layer, epoch, alpha):
  parameters = initialize_Xavier_parameter(neurons_per_layer)
  lost_list = []
  m = X_train.shape[0]

  #epoch번 경사하강을 한다
  for i in range(epoch):
    parameters.copy = parameters.copy()

    for x, y, in zip(X_train, Y_train):
      prediction, cache = forward_propagation(X, parameters)
      grads = backward_propagation(AL, Y, caches)
      parameters_copy = update(parameters_copy, grads, learning_rate)

      parameters = parameters_copy
      loss_list.append(compute_loss(X_train, Y_train, parameters))

  return loss_list, parameters